In [ ]:
/content/training_data_arrange_A001.npy

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Resizing
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt
tf.config.run_functions_eagerly(True)

In [3]:
def upsample_100(data):
    current_N = np.shape(data)
    new_data = np.zeros(shape=(100,current_N[1],current_N[2]))
    for i in range(0,100):
        new_data[i,:,:] = data[int(np.floor((current_N[0]*i)/100)),:,:]
    return new_data

def DFS():
    DFS_indexlist = np.array([2, 21, 9, 10,  11,  12, 25,  12,  24,  12,  11, 10, 9,
                             21, 3, 4, 3, 21, 5, 6, 7, 8, 23, 8, 22, 8, 7, 6, 5, 21,
                             2, 1, 13, 14, 15, 16, 15, 14, 13, 1, 17, 18, 19, 20, 19,
                             18, 17, 1, 2])
    DFS_indexlist = DFS_indexlist-1
    return DFS_indexlist

# Nicely formatted time string

def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# paths and variables
# main path:
path_folder = '/content/'

# action folder:
# only one action.
action_folder = 'A001'  

# file_name='S001C001P001R002A001.skeleton.npy' example.


DATA_PATH=os.path.join(path_folder,action_folder)
DATA_PATH_A=os.path.join(path_folder)
print(f"your data path: {DATA_PATH}")

#loading the training data.
training_binary_path = os.path.join(DATA_PATH_A,f'training_data_arrange_{action_folder}.npy')
print(training_binary_path)

your data path: /content/A001
/content/training_data_arrange_A001.npy


In [7]:
print("Loading previous training pickle...")
training_data = np.load(training_binary_path)

Loading previous training pickle...


In [14]:
def build_image_resizer(height,width):
    model = Sequential()
    model.add(Resizing(64, 64, interpolation="bilinear" , input_shape=(height,width,3)))
    return model

def build_generator(seed_size):
    # seed_size is z+c where c is the class sagment
    model = Sequential()
    
    #input layer
    model.add(Dense(4*4*512,activation="relu",input_dim=seed_size)) 
    model.add(Reshape((4,4,512)))

    # Mid CNN layer
    model.add(UpSampling2D(size=2,interpolation = 'bilinear'))
    model.add(Conv2D(512,kernel_size=(3,3),strides = (1, 1),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(Activation("relu"))

    model.add(UpSampling2D(size=(2, 2),interpolation = 'bilinear'))
    model.add(Conv2D(256,kernel_size=(3,3),strides = (1, 1),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D(size=(2, 2),interpolation = 'bilinear'))
    model.add(Conv2D(128,kernel_size=(3,3),strides = (1, 1),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D(size=(2, 2),interpolation = 'bilinear'))
    model.add(Conv2D(64,kernel_size=(3,3),strides = (1, 1),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(Activation("relu"))
    
    # Final CNN layer
    model.add(UpSampling2D(size=(2, 2),interpolation = 'bilinear'))
    model.add(Conv2D(3,kernel_size=(3,3),strides = (1, 1),padding="same",use_bias = False))
    model.add(Activation("tanh"))
    model.add(Resizing(64, 64, interpolation="bilinear" , input_shape=(128,128,3)))
    return model


def build_discriminator(image_shape):
    model = Sequential()
    
    #input layer first using resizer
    model.add(Conv2D(64, kernel_size=(4,4), strides=(2,2), input_shape=image_shape, padding="same",use_bias = False))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(128, kernel_size=(4,4), strides=(2,2),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, kernel_size=(4,4), strides=(2,2),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2D(512, kernel_size=(4,4), strides=(2,2),padding="same",use_bias = False))
    model.add(BatchNormalization(epsilon=10**-5,momentum=0.1))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [15]:
generator = build_generator(106)
discriminator = build_discriminator((64,64,9))
resizer = build_image_resizer(100,49)
Model.summary(resizer)
Model.summary(generator)
Model.summary(discriminator)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 64, 64, 3)         0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8192)              876544    
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 512)         0         
                                                                 
 up_sampling2d_5 (UpSampling  (None, 8, 8, 512)        0         
 2D)                                                             
                                      

In [30]:
#traning module
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def add_class(images):
    images = images.numpy()
    s = images.shape
    zero = np.zeros((s[0], s[1], s[2], 5))  
    one = np.ones((s[0], s[1], s[2], 1)) 
    class1 = np.append(zero, one, axis=3)
    return  np.append(images, class1, axis=3)
    

generator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)

# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images,seed):
    generator_resizer = build_image_resizer(128,128)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape: 
        generated_images = generator(seed, training=True)
        generated_images = tf.concat([generated_images,tf.zeros((generated_images.shape[0],64,64,5)), tf.ones((generated_images.shape[0],64,64,1))], 3)
        #generated_images = generator_resizer(generated_images, training=False)
        #proto_tensor = tf.make_tensor_proto(generated_images)
        #generated_images = tf.make_ndarray(proto_tensor)
        
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
    
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables),
    experimental_aggregate_gradients=False)
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables),
    experimental_aggregate_gradients=False)
    return gen_loss,disc_loss

def train(dataset, epochs):
    seed = np.random.normal(0, 1,100)
    c = np.asarray((0,0,0,0,0,1))
    fixed_seed = np.append(seed,c)
    
    seed = np.random.normal(0, 1,(BATCH_SIZE,100))
    s = seed.shape
    zero = np.zeros((s[0], 5))  
    one = np.ones((s[0], 1)) 
    class1 = np.append(zero, one, axis=1)
    seed = np.append(seed, class1, axis=1)
    
    
    start = time.time()

    for epoch in range(epochs):
        epoch_start = time.time()

        gen_loss_list = []
        disc_loss_list = []
        #i=0
        for image_batch in dataset:
            t = train_step(image_batch,seed)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])
            #i = i+1
            #print('image_batch'+str(i))

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)

        epoch_elapsed = time.time()-epoch_start
        print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'f' {hms_string(epoch_elapsed)}')
        #save_images(epoch,fixed_seed)

    elapsed = time.time()-start
    print (f'Training time: {hms_string(elapsed)}')



In [11]:
training_data_r = resizer(training_data, training=False)
print(training_data_r.shape)
s = training_data_r.shape
zero = np.zeros((s[0], s[1], s[2], 5))  
one = np.ones((s[0], s[1], s[2], 1)) 
class1 = np.append(zero, one, axis=3)
training_data_r1 = np.append(training_data_r, class1, axis=3)
print(training_data_r1.shape)


(939, 64, 64, 3)
(939, 64, 64, 9)


In [ ]:
EPOCHS = 50
BATCH_SIZE = 32
BUFFER_SIZE = 60000
SEED_SIZE = 106
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(training_data_r1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
#print(train_dataset.size)
train(train_dataset, EPOCHS)

Epoch 1, gen loss=1.554563283920288,disc loss=0.9927493929862976, 0:00:09.83
Epoch 2, gen loss=1.41997230052948,disc loss=1.0119458436965942, 0:00:09.63
Epoch 3, gen loss=1.0515707731246948,disc loss=1.3734499216079712, 0:00:10.32
Epoch 4, gen loss=0.9107743501663208,disc loss=1.3108322620391846, 0:00:09.58
Epoch 5, gen loss=1.0469051599502563,disc loss=1.16452956199646, 0:00:09.57
Epoch 6, gen loss=1.2579561471939087,disc loss=1.1443970203399658, 0:00:09.59
Epoch 7, gen loss=1.6970362663269043,disc loss=0.8603763580322266, 0:00:09.56
Epoch 8, gen loss=1.4362508058547974,disc loss=1.0911551713943481, 0:00:09.56
Epoch 9, gen loss=1.4579707384109497,disc loss=1.119246006011963, 0:00:09.69
Epoch 10, gen loss=1.7963359355926514,disc loss=0.7587465643882751, 0:00:09.57
Epoch 11, gen loss=1.6800841093063354,disc loss=0.9964626431465149, 0:00:09.56
Epoch 12, gen loss=2.103480815887451,disc loss=0.6586953401565552, 0:00:09.57
Epoch 13, gen loss=1.6975419521331787,disc loss=0.968756914138794, 0